In [ ]:
#20191007 이희원

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pylab as plt
import dmba
from dmba import plotDecisionTree, classificationSummary, regressionSummary
from sklearn import svm
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
!pip install --upgrade matplotlib
!pip install seaborn
import matplotlib.pyplot as plt

no display found. Using non-interactive Agg backend


In [2]:
chess_df_dt = pd.read_csv('data/chess_games.csv')
chess_df_rf = pd.read_csv('data/chess_games.csv')
chess_df_svm = pd.read_csv('data/chess_games.csv')
#데이터를 3개로 나눠서 진행하는 이유 : 각각 데이터의 내용이 달라질 예정이라 따로따로 진행
#3개 다 필요한 전처리 과정은 한번에 진행

In [ ]:
#일부 뽑아보기
print(chess_df_dt.head(10))

In [ ]:
#결측치 분석
chess_df_dt.info()

print(chess_df_dt.isnull().sum())
#opening_response와 opening_variation이 무엇인지 알아야함

In [3]:
#데이터 전처리 및 정규화
#결측치 처리하기
# opening_response -> 결측치를 'Unknown'으로 채움
chess_df_dt['opening_response'] = chess_df_dt['opening_response'].fillna('Unknown')
chess_df_rf['opening_response'] = chess_df_rf['opening_response'].fillna('Unknown')
chess_df_svm['opening_response'] = chess_df_svm['opening_response'].fillna('Unknown')

In [ ]:
# opening_variation은 뭘로 채울 것인가?
# 출력해보기
print(chess_df_dt['opening_variation'])

In [ ]:
#데이터 프레임에서 중복 제거, unique한 값 확인
chess_df_dt['opening_variation'].unique()

In [ ]:
#value_counts()로 개수 확인하기
chess_df_dt['opening_variation'].value_counts()

In [ ]:
#개방전의 variation의 최빈값으로 채우기
chess_df_dt['opening_variation'] = chess_df_dt['opening_variation'].fillna('#2')
chess_df_rf['opening_variation'] = chess_df_rf['opening_variation'].fillna('#2')
chess_df_svm['opening_variation'] = chess_df_svm['opening_variation'].fillna('#2')

In [ ]:
#데이터 탐색
#1. opening_name을 short나 full 중 하나만 남기기(full 삭제)
chess_df_dt = chess_df_dt.drop(columns = ['opening_fullname'])
chess_df_rf = chess_df_dt.drop(columns = ['opening_fullname'])
chess_df_svm = chess_df_dt.drop(columns = ['opening_fullname'])

In [ ]:
chess_df_dt.info() #fullname 삭제 완료

In [ ]:
#heatmap 그리기
corr = chess_df_dt[['rated', 'turns', 'white_rating', 'black_rating']].corr()

In [ ]:
#heatmap 그리기
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', center=0, ax=ax)
plt.show()
#white_rating과 black_rating이 서로 높은 상관관계 가짐

In [ ]:
#RandomForest

In [ ]:
#변수 추출하기 -> 10개로 축소
selected_columns = ['winner', 'moves', 'time_increment', 'rated', 'victory_status', 'turns', 'black_rating', 
                    'white_rating', 'victory_status', 'opening_moves']
chess_df_rf = chess_df_rf[selected_columns]

In [ ]:
#encoding(숫자형 범수가 아닌 것들을 인코딩 필요함)
#Winner
#time_increment
#rated
#victory_status

In [ ]:
#time_increment
# 문자열 열을 숫자로 변환 (time_increment 열)
chess_df_rf['time_increment'] = chess_df_rf['time_increment'].str.extract('(\d+)').astype(int)


In [ ]:
# 원핫인코딩을 위한 데이터 변환
chess_df_rf = pd.get_dummies(chess_df_rf)

# 결과 확인
print(chess_df_rf.head())

In [ ]:
X = chess_df_rf.drop(columns=['winner_Black', 'winner_Draw', 'winner_White'])
y = chess_df_rf[['winner_Black', 'winner_Draw', 'winner_White']]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=42)
RF = RandomForestClassifier(n_estimators=500, random_state=42)
RF.fit(train_X, train_y)

In [ ]:
rf_importances = RF.feature_importances_
std = np.std([tree.feature_importances_ for tree in RF.estimators_], axis=0)

df = pd.DataFrame({'feature' : train_X.columns, 'importance' : rf_importances, 'std' : std})
df = df.sort_values('importance')
print(df)

In [ ]:
#막대 그래프
ax = df.plot(kind='barh', xerr='std', x='feature', legend=False)
ax.set_ylabel('graph')

plt.tight_layout()
plt.show()

In [ ]:
# 실제값과 예측값을 비교하여 정확도 계산
accuracy = accuracy_score(valid_y, RF.predict(valid_X))

# 정확도 출력
print("Accuracy:", accuracy)

In [ ]:
#SVM

In [4]:
selected_columns = ['winner', 'moves', 'time_increment', 'rated', 'victory_status', 
                    'turns', 'black_rating', 
                    'white_rating', 'victory_status', 'opening_moves']
chess_df_svm = chess_df_svm[selected_columns]

In [ ]:
#수치형 변수 정규화하기

In [5]:
# 정규화를 수행할 변수들 선택
columns_to_normalize = ['turns', 'white_rating', 'black_rating']

# Min-Max 정규화 객체 생성
scaler = MinMaxScaler()

# 정규화 수행
normalized_data = scaler.fit_transform(chess_df_svm[columns_to_normalize])

In [ ]:
#나머지 정규화(time_increment)

In [6]:
#time_increment
# 문자열 열을 숫자로 변환 (time_increment 열)
chess_df_svm['time_increment'] = chess_df_svm['time_increment'].str.extract('(\d+)').astype(int)


In [ ]:
#one-hot encoding
#chess_df_svm = pd.get_dummies(chess_df_svm)

# 결과 확인
#print(chess_df_svm.head())

In [ ]:
#svm 적용하기

In [7]:
X = np.array(chess_df_svm[['time_increment', 'rated', 'turns'
                           , 'black_rating', 'white_rating', 'opening_moves']])
y = np.array(chess_df_svm['winner'])

clf = svm.SVC(kernel='linear', C=0.01)
clf.fit(X,y)

SVC(C=0.01, kernel='linear')

In [8]:
#성능평가
# 변수 중요도 확인
importance = np.abs(clf.coef_[0])
feature_names = ['time_increment', 'rated', 'turns', 'black_rating', 'white_rating', 'opening_moves']
feature_importance = dict(zip(feature_names, importance))
sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
print("Variable Importance:")
for feature, importance in sorted_importance:
    print(f"{feature}: {importance}")

# 정확도 측정
accuracy = clf.score(X, y)
print("Accuracy:", accuracy)

Variable Importance:
rated: 0.00032256099277816475
opening_moves: 5.363521793810833e-06
turns: 4.4159235130791785e-06
time_increment: 6.924663864538161e-07
white_rating: 4.5096385292708874e-08
black_rating: 2.964952727779746e-08
Accuracy: 0.6240402831787816
